In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

DATA_PATH = "/content/drive/MyDrive/the_datas/data/"

SEED = 42 # 시드값

# 데이터 블러오기
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

#공통 피처 파일 불러오기
train_ft = pd.read_csv(f"{DATA_PATH}train_common_4.csv") # 학습 데이터(피처)
test_ft = pd.read_csv(f"{DATA_PATH}test_common_4.csv") # 테스트 데이터(피처)

train_tr.shape , train_target.shape , test_tr.shape , submit.shape, train_ft.shape , test_ft.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2), (14940, 450), (12225, 450))

In [ ]:
target = train_target["target"]
target

,target
0,1.0
1,1.0
2,0.0
3,0.0
4,0.0
...,...
14935,0.0
14936,0.0
14937,0.0
14938,1.0


# AutoML(Automated machine learning)
- 시간 소모적이고 반복적인 기계 학습 모델 개발 작업을 자동화하는 프로세스
- 데이터 과학자, 분석가 및 개발자는 모델 품질을 유지하면서 확장성, 효율성 및 생산성이 높은 ML 모델을 빌드할 수 있다.

## FLAML(A Fast Library for Automated Machine Learning & Tuning)
- 머신러닝 학습 및 하이퍼파라미터 튜닝을 자동화해 주는 라이브러리
- https://microsoft.github.io/FLAML/docs/Use-Cases/Task-Oriented-AutoML
- flaml 설치하기
    ```bash
    pip install flaml
    ```

In [ ]:
pip install flaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 5.9 MB/s eta 0:00:00


- `AutoML` 클래스의 `fit` 메서드 주요 파라미터
    - metric
        - 평가지표
        - ex) 'roc_auc'
    - task
        - 작업 유형
        - ex) 'classification'
    - estimator_list
        - FLAML에서 제공해주는 모델들의 별칭을 리스트에 넣어주면 된다.
        - 생략시 자동으로 모델들이 선택된다.
        - ex) ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
    - time_budget
        - 수행 시간
            - 초단위
    - ensemble
        - 튜닝 후 스태킹 앙상블 여부
        - False(기본값) : 최상의 모델을 선택해서 학습
        - True: 스태킹하여 앙상블
        - `dict` 예시
            - {'final_estimator' : 사이킷런 모델 객체 }
    - n_splits
        - 교차검증 폴드 수(기본 5)
    - seed
        - 시드값
    - early_stop
        - True or False(기본값)
        - 튜닝이 수렴할 경우 조기 중지여부

In [ ]:
from flaml import AutoML
automl = AutoML()

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
%pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.6 MB/s eta 0:00:00


In [ ]:
params = {
    "metric": 'macro_f1',
    "task": 'classification',
    #"estimator_list" : ['xgboost', 'xgb_limitdepth', 'rf', 'lgbm', 'lgbm_spark', 'rf_spark', 'lrl1', 'lrl2', 'catboost', 'extra_tree', 'kneighbor', 'transformer', 'transformer_ms', 'histgb', 'svc', 'sgd', 'nb_spark', 'enet', 'lassolars', 'glr_spark', 'lr_spark', 'svc_spark', 'gbt_spark', 'aft_spark'],
    "estimator_list" : ['histgb','catboost', 'lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1'],
    "time_budget": 60*60,
    "seed" : SEED,
    "early_stop": True,
    # "ensemble" : True
}
automl.fit(train_ft, target, **params)


[flaml.automl.logger: 11-13 13:13:24] {1728} INFO - task = classification
[flaml.automl.logger: 11-13 13:13:24] {1739} INFO - Evaluation method: cv
[flaml.automl.logger: 11-13 13:13:25] {1838} INFO - Minimizing error metric: 1-macro_f1
[flaml.automl.logger: 11-13 13:13:25] {1955} INFO - List of ML learners in AutoML Run: ['histgb', 'catboost', 'lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl.logger: 11-13 13:13:25] {2258} INFO - iteration 0, current learner histgb
[flaml.automl.logger: 11-13 13:13:37] {2393} INFO - Estimated sufficient time budget=127411s. Estimated necessary time budget=3141s.
[flaml.automl.logger: 11-13 13:13:37] {2442} INFO -  at 17.2s,	estimator histgb's best error=0.6223,	best estimator histgb's best error=0.6223
[flaml.automl.logger: 11-13 13:13:37] {2258} INFO - iteration 1, current learner histgb
[flaml.automl.logger: 11-13 13:13:48] {2442} INFO -  at 27.9s,	estimator histgb's best error=0.6223,	best estimator histgb's best error=0

INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune


[flaml.automl.logger: 11-13 14:14:34] {2442} INFO -  at 3673.7s,	estimator lrl1's best error=0.6222,	best estimator xgboost's best error=0.2797
[flaml.automl.logger: 11-13 14:15:55] {2685} INFO - retrain xgboost for 81.2s
[flaml.automl.logger: 11-13 14:15:55] {2688} INFO - retrained model: XGBClassifier(base_score=None, booster=None, callbacks=[],
              colsample_bylevel=0.8411207958761456, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy='lossguide', importance_type=None,
              interaction_constraints=None, learning_rate=0.017361801680067852,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=0, max_leaves=4,
              min_child_weight=1.3061479700751337, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_e

- 선택된 모델 객체 확인

In [ ]:
automl.model.estimator

XGBClassifier(base_score=None, booster=None, callbacks=[],
              colsample_bylevel=0.8411207958761456, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy='lossguide', importance_type=None,
              interaction_constraints=None, learning_rate=0.017361801680067852,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=0, max_leaves=4,
              min_child_weight=1.3061479700751337, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=2519,
              n_jobs=-1, num_parallel_tree=None, random_state=None, ...)

- 튜닝된 하이퍼파라미터

In [ ]:
automl.best_config

{'n_estimators': 2519,
 'max_leaves': 4,
 'min_child_weight': 1.3061479700751337,
 'learning_rate': 0.017361801680067852,
 'subsample': 0.8539985606622676,
 'colsample_bylevel': 0.8411207958761456,
 'colsample_bytree': 1.0,
 'reg_alpha': 0.004074821343348109,
 'reg_lambda': 6.547882344386646}

- 선택된 모델의 cv 점수 확인하기

In [ ]:
1 - automl.best_loss

0.7202511492829202

- 예측 가능

In [ ]:
pred = automl.predict(test_ft)
pred

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
pred_proba = automl.predict_proba(test_ft) #여기서 실수해서 앙상블 파일에서 후처리함
pred_proba

array([[0.9032815 , 0.09671848],
       [0.61545503, 0.38454494],
       [0.70975727, 0.29024273],
       ...,
       [0.95313144, 0.04686856],
       [0.8543746 , 0.14562543],
       [0.9727974 , 0.02720259]], dtype=float32)

# 정답 데이터

In [ ]:
submit

,ID,target
0,test_0,0.5
1,test_1,0.5
2,test_2,0.5
3,test_3,0.5
4,test_4,0.5
...,...,...
12220,test_12220,0.5
12221,test_12221,0.5
12222,test_12222,0.5
12223,test_12223,0.5


In [ ]:
submit["target"] = pred
submit

,ID,target
0,test_0,0.0
1,test_1,0.0
2,test_2,0.0
3,test_3,1.0
4,test_4,1.0
...,...,...
12220,test_12220,1.0
12221,test_12221,0.0
12222,test_12222,0.0
12223,test_12223,0.0


#업로드

In [ ]:
submit.to_csv(f"{DATA_PATH}submit_새출발_automl_3.csv",index=False)